<a href="https://colab.research.google.com/github/hohohohyun/django/blob/master/%ED%95%9C%EA%B5%AD%EC%96%B4_LDA_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Kkma, Okt

In [69]:
!pip install konlpy

네이버영화 리뷰 데이터를 이용한 토픽 모델링 예시 입니다.

In [70]:
mr = pd.read_csv('/content/drive/My Drive/movie_review.txt', sep='\t', quoting=3)

In [71]:
mr

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0
...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0


In [72]:
## 문장 데이터만 사용
ex_data = mr['document']

In [73]:
#빈 문자열 NAN 값으로 바꾸기
clean_Data = ex_data.replace({'': np.nan})
clean_Data = clean_Data.replace(r'^\s*$', None, regex=True)

#한글이 아닌 단어 공백으로 처리
clean_Data = clean_Data.replace('^ ㄱ-힣', '', regex=True)

#NAN 이 있는 행은 삭제
clean_Data.dropna(how='any', inplace=True)

#행을 제거하면서 망가진 인덱스 초기화
clean_Data = clean_Data.reset_index (drop = True)

#데이터 프레임에 null 값이 있는지 확인
print(clean_Data.isnull().values.any()) 

False


In [74]:
from sklearn.decomposition import LatentDirichletAllocation

####형태소 분석기 꼬꼬마 사용



In [75]:
## 불용어 설정은 띄어쓰기로 구분하여 설정하시면 됩니다
## 기존에 설정되어있는 불용어 사전을 사용하려다가 토픽분석을 실시하면서 나오는 불용어들을 추가하면서 만들었습니다. 기존에 존재하는 한국어 불용어사전을 사용하셔도 됩니다.
## 한국어 불용어 사전 링크 https://www.ranks.nl/stopwords/korean

stop_words = '''ㅋㅋ 너무 진짜 정말 ㅎㅎ 같은 그냥 ㅋㅋㅋ 진짜 역시 그럼 근데 이거 하세요 ㅡㅡ ㅋㅋㅋㅋ ㅠㅠ 에서 한테 으로 하고 인데 원래 인가 이나 ㅉㅉ 하는 이다 하냐 해도 이랑 했는데 건가 합시다 하나 에는 하네 보고 보다 입니다 인지 이지 하면
이건 ㅋㅋㅋㅋㅋ 되는 이네 하지 이고 40 봤습니다 10 하다 이건 봤는데 까지 던데 ㄴ데 내가 우리'''

stop_words=stop_words.split(' ') ## 띄어쓰기로 구분된 불용어 리스트로 재설정(하나씩 따옴표 써가면서 만들기가 번거로워서 선택한 방법입니다.)

In [76]:
from konlpy.tag import Kkma  

## Kkma를 이용한 한국어 토큰화 과정

def tokenizing(tag, data):
  ## 선택한 품사tag 방식에 따른 토큰화 과정
  tag_=tag()
  tk_data = []
  for tkns in data:  
    tk_data.append(tag_.morphs(tkns))

  ## 불용어 제거단계  
  result = [] 
  for tkns in tk_data:
    tmp = []
    for w in tkns: 
      if w not in stop_words: 
          tmp.append(w) 
    result.append(tmp)
  
  return result

result = tokenizing(Kkma, clean_Data[-3000:]) ## 3000개로 예시

명사만 사용한 방법과 모든 단어를 다 사용한 방법을 둘다 사용하기 위해서 함수를 하나 더 설정했습니다.

In [77]:
## TF-IDF 행렬을 만들기 위하여 다시 띄어쓰기로 구분하여 문장형식으로 재구성
def corpus(tokens):
  corp = []
  for sent in tokens:
    tmp = " ".join(sent)
    corp.append(tmp)

  return corp

train_corp = corpus(result)

In [78]:
## 키워드 분석을 위하여 명사들만 추출
def n_corpus(corpus):
  corp = []
  for sent in corpus:
    tmp = " ".join(kkma.nouns(sent)) # Kkma에서 제공하는 품사 태깅에서 명사들만 추출하여 문장을 재구성한다.
    corp.append(tmp)
  return corp

n_corp = n_corpus(train_corp)

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

## 명사 뿐이 아닌 다른 단어도 사용하기 위해서는 n_corp 대신 train_corp 사용
tfidfv = TfidfVectorizer().fit(n_corp)
tfidf_km = tfidfv.transform(n_corp).toarray()
# print(tfidfv.vocabulary_) ## 구성단어 사전

In [80]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=0,max_iter=1)

In [81]:
lda_top=lda_model.fit_transform(tfidf_km)

In [82]:
terms = tfidfv.get_feature_names() ## TF-IDF vectorizer 안에 있는 단어들

def get_topics(components, feature_names, n=10): ## n : 토픽 별로 나타나는 단어의 갯수
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

## 결과 도출(Kkma)
get_topics(lda_model.components_,terms)

Topic 1: [('재미', 15.33), ('생각', 14.69), ('시간', 14.21), ('영화', 12.43), ('사랑', 10.37), ('기분', 9.72), ('추천', 8.12), ('기억', 6.63), ('노래', 6.43), ('다가', 5.48)]
Topic 2: [('영화', 116.85), ('최고', 37.03), ('연기', 33.22), ('미있', 29.14), ('재미있', 29.12), ('드라마', 26.13), ('사람', 23.82), ('감독', 22.77), ('내용', 22.69), ('배우', 21.58)]
Topic 3: [('재밌', 69.07), ('영화', 26.77), ('평점', 17.32), ('기대', 15.86), ('감동', 12.94), ('실망', 11.08), ('감동적', 9.51), ('세상', 8.78), ('신선', 8.1), ('가족', 7.92)]
Topic 4: [('버리', 10.1), ('영화', 8.77), ('이름', 8.36), ('공포', 7.32), ('장르', 7.28), ('걸작', 6.68), ('만화', 6.28), ('지리', 6.25), ('강추', 6.05), ('전쟁', 5.92)]
Topic 5: [('미없', 27.14), ('재미없', 26.87), ('이것', 23.98), ('영화', 20.03), ('가슴', 11.45), ('이상', 10.54), ('수준', 10.4), ('름답', 7.53), ('짜증', 7.46), ('평가', 6.43)]


#### 형태소 분석기 Okt(Open Korea Text) 사용
 - 위와 동일하고 형태소 분석기만 Okt를 사용하였습니다.

In [83]:
from konlpy.tag import Okt  

result_o = tokenizing(Okt, clean_Data[-3000:]) ## 3000개로 예시

In [84]:
train_corp_o = corpus(result_o)

In [85]:
n_corp_o = n_corpus(train_corp_o)

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfv = TfidfVectorizer().fit(n_corp_o)
tfidf_ = tfidfv.transform(n_corp_o).toarray()
# print(tfidfv.vocabulary_[:100])

In [87]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,learning_method='online',random_state=0,max_iter=1)

In [88]:
lda_top=lda_model.fit_transform(tfidf_)

In [89]:
terms = tfidfv.get_feature_names() ## TF-IDF vectorizer 안에 있는 단어들

def get_topics(components, feature_names, n=10): ## n : 토픽 별로 나타나는 단어의 갯수
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

## 결과 도출(Okt)
get_top ics(lda_model.components_,terms)

Topic 1: [('영화', 22.54), ('최악', 19.76), ('재밋', 13.81), ('인생', 12.28), ('주인공', 10.56), ('사람', 10.21), ('진심', 10.11), ('아이', 9.25), ('나름', 8.61), ('무엇', 7.91)]
Topic 2: [('재미', 31.58), ('드라마', 20.84), ('완전', 16.89), ('흥미', 10.21), ('명작', 9.36), ('현실', 7.02), ('마무리', 6.75), ('모습', 6.54), ('영화', 6.24), ('내용', 5.56)]
Topic 3: [('최고', 43.51), ('영화', 43.37), ('평점', 33.56), ('생각', 30.63), ('쓰레기', 20.4), ('이야기', 19.59), ('감독', 17.83), ('사랑', 14.69), ('이상', 12.86), ('작품', 12.27)]
Topic 4: [('감동', 37.87), ('영화', 35.92), ('연기', 26.21), ('스토리', 23.15), ('배우', 17.6), ('처음', 15.51), ('내용', 15.12), ('기대', 14.97), ('정도', 14.95), ('가슴', 13.69)]
Topic 5: [('영화', 84.16), ('시간', 25.66), ('코미디', 13.48), ('매력', 10.88), ('후회', 9.88), ('음악', 9.62), ('가족', 8.92), ('추천', 8.49), ('수준', 8.27), ('장르', 8.15)]
